In [2]:
import httpx
import time
import pandas as pd

In [3]:
## Functions I might need later:

# def dedupe(xs):
#     return list(dict.fromkeys([x for x in xs if x]))

# env_post_ids = dedupe(env_post_ids)
# infr_post_ids = dedupe(infr_post_ids)
# housing_post_ids = dedupe(housing_post_ids)
# econ_post_ids = dedupe(econ_post_ids)
# life_qual_post_ids = dedupe(life_qual_post_ids)
# aesth_post_ids = dedupe(aesth_post_ids)
# gov_post_ids = dedupe(gov_post_ids)
# not_useful_post_ids = dedupe(not_useful_post_ids)

In [4]:
MAX_POSTS = 500

all_post_ids = []

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "Virginia"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

after = None

environmental_keywords = ["pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["electricity", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "approval", "permit", "city council"]

# NEED TO INCLUDE THIS FILTERING
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    while True:
        if after is not None:
            params["after"] = after
        else:
            params.pop("after", None)

        r = client.get(search_url, params=params)
        print("REQUEST URL:", str(r.url))
        r.raise_for_status()
        listing = r.json()

        children = listing["data"]["children"]
        if not children:
            break

        for child in children:
            # data = child.get("data", {})
            # post_id = data.get("id")

            # title = (data.get("title") or "")
            # selftext = (data.get("selftext") or "")
            # text = (title + "\n" + selftext).lower()

            post_id = child["data"]["id"]
            all_post_ids.append(post_id)
            title = (child["data"]["title"] or "")
            selftext = (child["data"]["selftext"] or "")
            text = (title + " " + selftext).lower()

            matched = False
        
            if any(kw in text for kw in environmental_keywords):
                env_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in infrastructure_utilities_keywords):
                infr_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in housing_keywords):
                housing_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in economic_keywords):
                econ_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in life_quality_keywords):
                life_qual_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in aesthetics_keywords):
                aesth_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in government_keywords):
                gov_post_ids.append(post_id)
                matched = True
            if not matched:
                not_useful_post_ids.append(post_id)
            
        after = listing["data"]["after"]
        print("After:", after)
        if after is None:
            break

        if len(all_post_ids) >= MAX_POSTS:
            break

        time.sleep(0.2)

print("Total posts scanned:", len(all_post_ids))
print("Found environmental posts:", len(env_post_ids))
print("Found infrastructure posts:", len(infr_post_ids))
print("Found housing posts:", len(housing_post_ids))
print("Found economic posts:", len(econ_post_ids))
print("Found life quality posts:", len(life_qual_post_ids))
print("Found aesthetic posts:", len(aesth_post_ids))
print("Found government posts:", len(gov_post_ids))

REQUEST URL: https://www.reddit.com/r/Virginia/search.json?q=datacenter&restrict_sr=1
After: None
Total posts scanned: 10
Found environmental posts: 1
Found infrastructure posts: 3
Found housing posts: 2
Found economic posts: 3
Found life quality posts: 1
Found aesthetic posts: 2
Found government posts: 3


In [5]:
#listing -> data -> children (only 1 when going to a specific post) -> data -> selftext/created/id
#comments:children - data - body

#comments have to be above 200 characters? - some that are under express good opinions so idk (maybe over 75?)

In [6]:
nova_theme_lists = [env_post_ids, infr_post_ids, housing_post_ids, econ_post_ids, life_qual_post_ids, aesth_post_ids, gov_post_ids]
posts = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])

for theme in nova_theme_lists:
    df1 = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])
    post_ids = []
    post_texts = []
    post_dates = []
    with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in theme:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)
            r.raise_for_status()
            post_json = r.json()
            post_ids.append(pid)
            text_and_title = post_json[0]["data"]["children"][0]["data"]["selftext"] + post_json[0]["data"]["children"][0]["data"]["title"]
            post_texts.append(text_and_title)
            post_dates.append(post_json[0]["data"]["children"][0]["data"]["created_utc"])
    df1["ids"] = post_ids
    df1["text"] = post_texts
    df1["date"] = post_dates
    if theme == env_post_ids:
        df1["environment"] = True
    if theme == infr_post_ids:
        df1["infrastructure"] = True
    if theme == housing_post_ids:
        df1["housing"] = True
    if theme == econ_post_ids:
        df1["economy"] = True
    if theme == life_qual_post_ids:
        df1["life quality"] = True
    if theme == aesth_post_ids:
        df1["aesthetics"] = True
    if theme == gov_post_ids:
        df1["government"] = True
    posts = pd.concat([posts, df1], ignore_index=True)


print(posts)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_54171/1697576501.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  posts = pd.concat([posts, df1], ignore_index=True)


        ids                                               text          date  \
0   1c0zjyr  Does that piss you off?  As a dumb rando that ...  1.712791e+09   
1    x977kr  Virginia has more datacenter capacity than Eur...  1.662660e+09   
2   1c0zjyr  Does that piss you off?  As a dumb rando that ...  1.712791e+09   
3   13yzjao  \nThe data center developer behind the Catlett...  1.685763e+09   
4   1j9k8hj  Data centers are changing the landscape. Here’...  1.741787e+09   
5   17vltwe  [https://www.insidenova.com/news/politics/prin...  1.700023e+09   
6   1kt067k  Democratic leadership signals willingness to r...  1.747944e+09   
7   1c0zjyr  Does that piss you off?  As a dumb rando that ...  1.712791e+09   
8   13yzjao  \nThe data center developer behind the Catlett...  1.685763e+09   
9   17vltwe  [https://www.insidenova.com/news/politics/prin...  1.700023e+09   
10  1j9k8hj  Data centers are changing the landscape. Here’...  1.741787e+09   
11  1c0zjyr  Does that piss you off?  As

In [7]:
grouping_cols = ["ids", "text", "date"]
posts = posts.groupby(grouping_cols, as_index=False).max()
theme_cols = ["environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"]
posts[theme_cols] = posts[theme_cols].fillna(False)
posts.head(15)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_54171/3906843801.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  posts[theme_cols] = posts[theme_cols].fillna(False)


,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government
0,13yzjao,\nThe data center developer behind the Catlett...,1.685763e+09,False,True,False,True,False,False,True
1,17vltwe,[https://www.insidenova.com/news/politics/prin...,1.700023e+09,False,False,True,False,True,False,True
2,1c0zjyr,Does that piss you off? As a dumb rando that ...,1.712791e+09,True,True,False,True,False,True,True
3,1j9k8hj,Data centers are changing the landscape. Here’...,1.741787e+09,False,False,True,False,False,True,False
4,1kt067k,Democratic leadership signals willingness to r...,1.747944e+09,False,False,False,True,False,False,False
5,x977kr,Virginia has more datacenter capacity than Eur...,1.662660e+09,False,True,False,False,False,False,False


In [8]:
import re
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512        
)

/Users/mnatali/Projects/sentiment_analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [9]:
texts = posts["text"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
posts["roberta"] = results

In [10]:
posts["name"] = posts["roberta"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
posts["label"] = posts["name"].apply(roberta_label)

posts["degree"] = posts["roberta"].apply(lambda x: x["score"])
posts = posts.drop(["roberta", "name"], axis=1)

In [11]:
posts.head(15)

,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government,label,degree
0,13yzjao,\nThe data center developer behind the Catlett...,1.685763e+09,False,True,False,True,False,False,True,neutral,0.844348
1,17vltwe,[https://www.insidenova.com/news/politics/prin...,1.700023e+09,False,False,True,False,True,False,True,negative,0.513290
2,1c0zjyr,Does that piss you off? As a dumb rando that ...,1.712791e+09,True,True,False,True,False,True,True,negative,0.646093
3,1j9k8hj,Data centers are changing the landscape. Here’...,1.741787e+09,False,False,True,False,False,True,False,negative,0.641380
4,1kt067k,Democratic leadership signals willingness to r...,1.747944e+09,False,False,False,True,False,False,False,neutral,0.565483
5,x977kr,Virginia has more datacenter capacity than Eur...,1.662660e+09,False,True,False,False,False,False,False,neutral,0.607460


In [12]:
## calculating average sentiment by theme
# Method: Weigh all posts by their degree in the numerator and denominator, means that the average sentiment will just be +/- 1 if there are only positive or negative themes but other than that does a pretty good job of weighing neutrality
def avg_sentiment_calculation(theme):
    theme_posts = posts[posts[theme] == True]
    pos = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum()
    neg = theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    total = theme_posts["degree"].sum()
    return (pos-neg)/total

print("Average sentiment of environmental posts:", avg_sentiment_calculation("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation("government"))

Average sentiment of environmental posts: -1.0
Average sentiment of infrastructural posts: -0.30797102293386996
Average sentiment of housing-related posts: -1.0
Average sentiment of economic posts: -0.31425904527398146
Average sentiment of life-quality-related posts: -1.0
Average sentiment of aesthetics-related posts: -1.0
Average sentiment of governmental posts: -0.5786119352637443
